# Configuracion-Instalacion de librerias

In [1]:
!pip install langchain==0.0.324 faiss-cpu==1.7.4 PyPDF2==3.0.1 sentence-transformers==2.2.2 python-dotenv==1.0.0 -q

  error: subprocess-exited-with-error
  
  × Building wheel for faiss-cpu (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      running bdist_wheel
      running build
      running build_py
      running build_ext
      building 'faiss._swigfaiss' extension
      swigging faiss\faiss\python\swigfaiss.i to faiss\faiss\python\swigfaiss_wrap.cpp
      swig.exe -python -c++ -Doverride= -I/usr/local/include -Ifaiss -doxygen -DSWIGWIN -module swigfaiss -o faiss\faiss\python\swigfaiss_wrap.cpp faiss\faiss\python\swigfaiss.i
      error: command 'swig.exe' failed: None
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for faiss-cpu
ERROR: Could not build wheels for faiss-cpu, which is required to install pyproject.toml-based projects


# Subir el modelo pdf

In [4]:
import requests
import os
from dotenv import load_dotenv
load_dotenv
URL= ("http://www.bnm.me.gov.ar/giga1/documentos/EL006510.pdf")
descarga = requests.get(URL)
pdf_file = open("EL006510.pdf", "wb")
pdf_file.write(descarga.content)

26843537

# Lectura del PDF

In [5]:
!pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [6]:
from PyPDF2 import PdfReader

pdf_file_obj = open("EL006510.pdf", "rb")
pdf_reader = PdfReader(pdf_file_obj)

In [7]:
text = ""

for page in pdf_reader.pages:
    text += page.extract_text()

muestra de lo que guardo en text:

In [8]:
print(text[26480:29998])

 favorecer los intercambios, permitieron el comercio entre colonias y alentaron el desarrollo de sus posesiones ubicadas en el litoral atlántico de América, durante mucho tiempo olvidadas, y muy codiciadas por sus rivales europeos.
Sin embargo, esta apertura de puertos y el aliento del comercio metrópoli-
colonias, que beneficiaba, entre otras, a la ciudad de Buenos Aires, transformada en capital de un nuevo virreinato, el Virreinato del Río de la Plata, constituyó una renovada fuente de conflictos. Sólo unos pocos comerciantes, todos ellos de origen español, administraban ese fortalecido monopolio. Eran dueños de importantes compañías comerciales y disponían de los contactos y redes comerciales que vinculaban a las colonias con las principales casas de comercio de España. Muchos, y sobre todo los comerciantes criollos, quedaron afuera de estos negocios. Recordemos que también, como criollos, eran excluidos de los más importantes cargos políticos, administrativos y eclesiásticos.
En el

# Creacion de los chunk

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
    )

In [11]:
chunks = text_splitter.split_text(text)

In [12]:
len(chunks)

789

In [13]:
print(chunks[13])

................162Lámina: Represión y neoliberalismo: de la dictadura de 1976 a la crisis de 2001  ............................................172unidad 6: reconquista de los derechos  y reconstrucción de la democracia  .........175La Argentina y el resto del mundo  en los inicios del siglo XXI  ........................176Tiempos de crisis: inestabilidad política  y represión  ................................................177Néstor Kirchner y un proyectode cambio  .................................................178La reconfiguración del Estado  .................180La política de derechos humanos  ............182El nuevo mapa regional ............................ 183Cristina Fernández de Kirchner y la profundización de un modelo  ...................184Crecimiento con inclusión social


# Creacion de embedding

In [16]:
!pip install sentence_transformers -q

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

c:\Users\abans\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [18]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

c:\Users\abans\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentence_embeddings = model.encode("Una revolución no es un hecho puntual aunque el concepto invita a pensaren cierta aceleración de hechos que terminan bruscamente con un estado de cosas para pasar a la construcción de un orden social nuevo, cuando hablamos de revolución nos referimos a procesos")

In [21]:
len(sentence_embeddings)

384

# Pregunta con el PDF

In [24]:
!pip install faiss-cpu -q

In [22]:
from langchain.vectorstores import FAISS

In [25]:
knowledge_base = FAISS.from_texts(chunks, embeddings)

Ejemplo 1


In [26]:
pregunta = "¿A qué llamamos revolucion?"
docs = knowledge_base.similarity_search(pregunta, 1)

In [30]:
print(docs)

[Document(page_content='Bibliografía  ...............................................206uNidad 1\nla revolución de mayo12 Historia arGENtiNa¿a qué llamamos rE volu CióN?\nMucho se ha escrito acerca de este concepto. En Ciencias Sociales se \nutiliza para referir a un cambio profundo en los aspectos económicos, políticos, \nsociales y culturales de una sociedad determinada.\nUna revolución no es un hecho puntual; aunque el concepto invita a pensar')]


Ejemplo 2

In [31]:
pregunta= "¿Que es  El iluminismo?"
docs= knowledge_base.similarity_search(pregunta,1)

In [32]:
print(docs)

[Document(page_content='en las que, durante la segunda mitad del siglo XVIII, tuvieron oportunidad de nutrirse en el pensamiento Ilustrado (o iluminista), un cuerpo de ideas que cuestionaba las estructuras basadas en la organización estamental, y los privilegios y derechos exclusivos de la nobleza. En fin, tuvieron la posibilidad de formarse en teorías que discutían la desigualdad y la tiranía de los más poderosos.')]
